# Препроцессинг, инжиниринг фич

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling as pp

In [237]:
train = pd.read_csv('train.csv', sep = '\t', index_col = 'Unnamed: 0')
test = pd.read_csv('test.csv', sep = '\t', index_col = 'Unnamed: 0')

In [8]:
train_report = pp.ProfileReport(train)
train_report.to_file(outputfile='train_rep.html')

In [7]:
test_report = pp.ProfileReport(test)
test_report.to_file(outputfile='test_rep.html')

фичи которые имеют корреляцию >= 0.95

In [34]:
test_report.get_rejected_variables(threshold = 0.95)

['122', '146', '148', '153', '165', '179', '180', '323', '326', '9']

In [33]:
train_report.get_rejected_variables(threshold = 0.95)

['146', '148', '153', '165', '180', '323', '326', '9']

константные фичи

In [86]:
const_list_train = [i for i in train.columns if len(train[i].unique()) == 1]
const_list_train

['140', '152', '160', '164']

In [61]:
const_list_test = [i for i in test.columns if len(test[i].unique()) == 1]
const_list_test

['0',
 '5',
 '11',
 '129',
 '130',
 '137',
 '138',
 '140',
 '141',
 '149',
 '150',
 '164',
 '178',
 '186',
 '188',
 '192',
 '193',
 '291',
 '301',
 '303',
 '305']

In [64]:
# check 152, 160 - constant in train, not in test 
print(test['152'].value_counts())
print(test['160'].value_counts())

0    4165
1       1
Name: 152, dtype: int64
0    4165
1       1
Name: 160, dtype: int64


In [85]:
# distribution of features in train which are constant in test 
for i in const_list_test[1:]:
    print(i)
    print(train[i].value_counts())
    print('target')
    print(train.loc[train[i] == np.argmin(train[i].value_counts()), '0'].value_counts(),'\n')
    

5
0    30481
1       19
Name: 5, dtype: int64
target
0    11
1     8
Name: 0, dtype: int64 

11
1    30495
0        5
Name: 11, dtype: int64
target
0    3
1    2
Name: 0, dtype: int64 

129
0    30489
1       11
Name: 129, dtype: int64
target
0    9
1    2
Name: 0, dtype: int64 

130
0    30498
1        2
Name: 130, dtype: int64
target
0    2
Name: 0, dtype: int64 

137
0    30477
1       23
Name: 137, dtype: int64
target
0    21
1     2
Name: 0, dtype: int64 

138
0    30498
1        2
Name: 138, dtype: int64
target
0    2
Name: 0, dtype: int64 

140
0    30500
Name: 140, dtype: int64
target
0    25077
1     5423
Name: 0, dtype: int64 

141
0    30498
1        2
Name: 141, dtype: int64
target
0    2
Name: 0, dtype: int64 

149
0    30491
1        9
Name: 149, dtype: int64
target
0    7
1    2
Name: 0, dtype: int64 

150
0    30497
1        3
Name: 150, dtype: int64
target
0    3
Name: 0, dtype: int64 

164
0    30500
Name: 164, dtype: int64
target
0    25077
1     5423
Name: 0, dtype:

# columns to delete 

1. all constant in train  ['140', '152', '160', '164']
2. 305, 303, 301, 193, 192, 186, 150, 141, 138, 130 - because of very high correlation

In [238]:
train.drop(['140', '152', '160', '164','305', '303', '301', '193', '192', '186', '150', '141', '138', '130'],inplace = True,axis = 1)
test.drop(['140', '152', '160', '164','305', '303', '301', '193', '192', '186', '150', '141', '138', '130'],inplace = True, axis = 1)

In [ ]:
X_train, y_train = train[train.columns[1:]], train['0']
X_test, y_test = test[test.columns[1:]], test['0'].fillna(0, inplace = True)

In [118]:
# names of columns with binary variables
binary_list = [i for i in X_train.columns if len(X_train[i].value_counts()) == 2]

In [179]:
# matthews correlation of binary columns with target variable

from sklearn.metrics import matthews_corrcoef
import operator

matt_corr = {}
high_negative = {}
high_positive = {}

for i in binary_list:
    matt_corr[i] = matthews_corrcoef(y_train.values, X_train[i].values)
    if matt_corr[i] >= 0.07:
        high_positive[i] = matthews_corrcoef(y_train.values, X_train[i].values)
    elif matt_corr[i] <= -0.07:
        high_negative[i] = matthews_corrcoef(y_train.values, X_train[i].values)
        
        
    
high_positive_sorted = sorted(high_positive.items(), key=operator.itemgetter(1))
high_negative_sorted = sorted(high_negative.items(), key=operator.itemgetter(1))    
matt_corr_sorted = sorted(matt_corr.items(), key=operator.itemgetter(1))
matt_corr_sorted

[('337', -0.1457217397960931),
 ('298', -0.12757933553894957),
 ('332', -0.12439091984025676),
 ('147', -0.079413411781676949),
 ('36', -0.078344089067329731),
 ('165', -0.064505470228960943),
 ('62', -0.063916152678570634),
 ('22', -0.063882876113892789),
 ('48', -0.055970430573814171),
 ('44', -0.051059659135367468),
 ('223', -0.045952782665356302),
 ('190', -0.045463305535124503),
 ('18', -0.037554344349653615),
 ('2', -0.036854772188801149),
 ('9', -0.036854772188801149),
 ('216', -0.033268998110934783),
 ('47', -0.030258608716411801),
 ('31', -0.030037252859205651),
 ('49', -0.029700379647389964),
 ('53', -0.026874343253007884),
 ('56', -0.026360318390569275),
 ('58', -0.025830823936705993),
 ('54', -0.025292763511370858),
 ('24', -0.024692106120664629),
 ('220', -0.023839802886202122),
 ('59', -0.023393387705996076),
 ('64', -0.023039815344711074),
 ('57', -0.02254661411898504),
 ('179', -0.022148312614710151),
 ('23', -0.022028425579064013),
 ('167', -0.021504518465117146),
 ('5

In [192]:
high_positive_sorted

[('297', 0.070316715621584813),
 ('283', 0.078104858174954203),
 ('219', 0.084407935505334042),
 ('65', 0.088044079785767143),
 ('60', 0.10857143016592991),
 ('4', 0.11745329253152549),
 ('34', 0.12237638784182032),
 ('144', 0.12303992048636103),
 ('295', 0.13226417112186684),
 ('338', 0.15552247882925146)]

In [217]:
high_negative_sorted

[('337', -0.1457217397960931),
 ('298', -0.12757933553894957),
 ('332', -0.12439091984025676),
 ('147', -0.079413411781676949),
 ('36', -0.078344089067329731)]

# Попытка сгенерировать новые фичи
Рассматриваю только те бинарные фичи, которые имеют достаточную позитивную и негативную корреляцию(>=0.07 и <=-0.07 соответственно) с таргетом 

Буду делать так: если корр. Мэтьюса двух фич между собой < 0.3 тогда я создаю новую фичу как их дизюнкцию, а если больше 0.6 - то как их конюнкцию. И если новая фича коррелирует со старыми с коэфф > 0.99 то я ее удаляю

( К сожалению, этот подход практически не давал прироста качества - поэтому в большинстве случаев я использовал датасеты без новых фич)

In [250]:
for i in range(len(list(high_positive.keys()))):
    for j in range(i+1 ,len(list(high_positive.keys()))):
        if matthews_corrcoef(X_train[list(high_positive.keys())[i]], X_train[list(high_positive.keys())[j]]) < 0.3:
            mix = np.logical_or(X_train[list(high_positive.keys())[i]], X_train[list(high_positive.keys())[j]])
            mix = np.array([int(i) for i in mix])
            if (matthews_corrcoef(mix, X_train[list(high_positive.keys())[i]]) < 0.99) and (matthews_corrcoef(mix, X_train[list(high_positive.keys())[i]])) < 0.99:
                X_train['mixOR '+list(high_positive.keys())[i]+'-'+list(high_positive.keys())[j]] = mix
                mix_1 = np.logical_or(X_test[list(high_positive.keys())[i]], X_test[list(high_positive.keys())[j]])
                mix_1 = np.array([int(i) for i in mix_1])                
                X_test['mixOR '+list(high_positive.keys())[i]+'-'+list(high_positive.keys())[j]] = mix_1


        if matthews_corrcoef(X_train[list(high_positive.keys())[i]], X_train[list(high_positive.keys())[j]]) > 0.6:
            mix = np.logical_and(X_train[list(high_positive.keys())[i]], X_train[list(high_positive.keys())[j]])
            mix = np.array([int(i) for i in mix])
            if (matthews_corrcoef(mix, X_train[list(high_positive.keys())[i]]) < 0.99) and (matthews_corrcoef(mix, X_train[list(high_positive.keys())[i]])) < 0.99:
                X_train['mixAND '+list(high_positive.keys())[i]+'-'+ list(high_positive.keys())[j]] = mix
                mix_1 = np.logical_and(X_test[list(high_positive.keys())[i]], X_test[list(high_positive.keys())[j]])
                mix_1 = np.array([int(i) for i in mix_1])      
                X_test['mixAND '+list(high_positive.keys())[i]+'-'+ list(high_positive.keys())[j]] = mix_1



            '''
            print(i,'  ',j,'   ',matthews_corrcoef(X_train[list(high_positive.keys())[i]], X_train[list(high_positive.keys())[j]]))
            print(matthews_corrcoef(mix, y_train.values))
            print(matthews_corrcoef(mix, X_train[list(high_positive.keys())[i]]))
            print(matthews_corrcoef(mix, X_train[list(high_positive.keys())[j]]))
            print(matthews_corrcoef(y_train.values, X_train[list(high_positive.keys())[i]]))
            print(matthews_corrcoef(y_train.values, X_train[list(high_positive.keys())[j]]))
            '''
            

In [251]:
for i in range(len(list(high_negative.keys()))):
    for j in range(i+1 ,len(list(high_negative.keys()))):        
        if matthews_corrcoef(X_train[list(high_negative.keys())[i]], X_train[list(high_negative.keys())[j]]) < 0.3:
            mix = np.logical_or(X_train[list(high_negative.keys())[i]], X_train[list(high_negative.keys())[j]])
            mix = np.array([int(i) for i in mix])
            if (matthews_corrcoef(mix, X_train[list(high_negative.keys())[i]]) < 0.99) and (matthews_corrcoef(mix, X_train[list(high_negative.keys())[i]])) < 0.99:
                X_train['mixOR '+list(high_negative.keys())[i]+'-'+list(high_negative.keys())[j]] = mix
                mix_1 = np.logical_or(X_test[list(high_negative.keys())[i]], X_test[list(high_negative.keys())[j]])
                mix_1 = np.array([int(i) for i in mix_1])                
                X_test['mixOR '+list(high_negative.keys())[i]+'-'+list(high_negative.keys())[j]] = mix_1


                
        if matthews_corrcoef(X_train[list(high_negative.keys())[i]], X_train[list(high_negative.keys())[j]]) > 0.6:
            mix = np.logical_and(X_train[list(high_negative.keys())[i]], X_train[list(high_negative.keys())[j]])
            mix = np.array([int(i) for i in mix])
            if (matthews_corrcoef(mix, X_train[list(high_negative.keys())[i]]) < 0.99) and (matthews_corrcoef(mix, X_train[list(high_negative.keys())[i]])) < 0.99:
                X_train['mixAND '+list(high_negative.keys())[i]+'-'+ list(high_negative.keys())[j]] = mix
                
                mix_1 = np.logical_and(X_test[list(high_negative.keys())[i]], X_test[list(high_negative.keys())[j]])
                mix_1 = np.array([int(i) for i in mix_1])                
                X_test['mixOR '+list(high_negative.keys())[i]+'-'+list(high_negative.keys())[j]] = mix_1


        

        '''    
        print(i,'  ',j,'   ',matthews_corrcoef(X_train[list(high_negative.keys())[i]], X_train[list(high_negative.keys())[j]]))
        print(matthews_corrcoef(mix, y_train.values))
        print(matthews_corrcoef(mix, X_train[list(high_negative.keys())[i]]))
        print(matthews_corrcoef(mix, X_train[list(high_negative.keys())[j]]))
        print(matthews_corrcoef(y_train.values, X_train[list(high_negative.keys())[i]]))
        print(matthews_corrcoef(y_train.values, X_train[list(high_negative.keys())[j]]))
        '''
            

начальная тестовая выборка

In [294]:
X_train_1, y_train_1 = train[train.columns[1:]], train['0']
X_test_1, y_test_1 = test[test.columns[1:]], test['0'].fillna(0, inplace = True)

Кросс-валидацию я использовал StratifiedKFold()